In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
#import yfinance as yf
# import tensorflow as tf
# import tensorflow.compat.v2.feature_column as fc
from IPython.display import clear_output
import pyodbc

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data_final, load_data_and_save_v2, strList
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing
# from utils.models import make_input_fn
# from utils.models import performanceTesting,scoreFunction

### Load data etc

In [4]:
stockInfo = pd.read_csv('../utils/stockInfo.csv',header=[0,1])

In [5]:
stockInfo.dropna()[stockInfo.dropna()[('2020-07-05','exchange')]=='NMS'].sort_values(('2020-07-05','marketCap'))

date              2020-07-05                       
     ticker                  sector exchange     marketCap
7671   TCCO              Technology      NMS  5.902776e+06
2323   EDRY             Industrials      NMS  8.550177e+06
5016   MDRR             Real Estate      NMS  8.736265e+06
4262   ISIG  Communication Services      NMS  9.193270e+06
5644   NSYS              Technology      NMS  1.063012e+07
...     ...                     ...      ...           ...
4181   INTC              Technology      NMS  2.503564e+11
2733     FB  Communication Services      NMS  6.650369e+11
3462   GOOG  Communication Services      NMS  1.001645e+12
5272   MSFT              Technology      NMS  1.564160e+12
12     AAPL              Technology      NMS  1.578173e+12

[1295 rows x 4 columns]

In [6]:
# How many are there contained in the file?
print('There are',
      stockInfo.shape[0],
      'tickers contained in the file, and there is data on',
      stockInfo.isnull().sum()[1:].min(),'of them.')

There are 8850 tickers contained in the file, and there is data on 5306 of them.


## Sneak peak on the 10 largest companies

In [7]:

stockInfo.sort_values(('2020-07-05','marketCap'),ascending=False).head(10)

date              2020-07-05                       
     ticker                  sector exchange     marketCap
12     AAPL              Technology      NMS  1.578173e+12
5272   MSFT              Technology      NMS  1.564160e+12
3462   GOOG  Communication Services      NMS  1.001645e+12
2733     FB  Communication Services      NMS  6.650369e+11
699    BABA       Consumer Cyclical      NYQ  5.936536e+11
8228      V      Financial Services      NYQ  4.298068e+11
4459    JNJ              Healthcare      NYQ  3.716463e+11
8583    WMT      Consumer Defensive      NYQ  3.375968e+11
4930     MA      Financial Services      NYQ  3.035511e+11
7950    TSM              Technology      NYQ  2.837483e+11

## How is the tickers divided in sectors?

In [8]:
stockInfo.loc[:,('2020-07-05')].groupby(['sector']).count()#rename(None,axis=1)

,exchange,marketCap
sector,,
Basic Materials,137,137
Communication Services,146,146
Consumer Cyclical,318,317
Consumer Defensive,130,130
Energy,204,204
Financial,3,3
Financial Services,942,942
Healthcare,607,607
Industrials,404,404


## Lets get the X largest companies in each sector.

In [9]:
sectors = stockInfo.loc[:,('2020-07-05')].dropna(axis=0).sector.unique()

X = 5

topXsectors = pd.DataFrame(index = np.arange(X),columns = pd.MultiIndex.from_product([['Top {}'.format(X)],sectors]))

t1 = pd.DataFrame({'tickers':stockInfo.loc[:,'date'].ticker.values})
t1[stockInfo.loc[:,'2020-07-05'].columns] = stockInfo.loc[:,'2020-07-05']

for i,sector in enumerate(sectors):
    
    tempSec = t1[t1.sector==sector].sort_values('marketCap',
                                                      ascending=False).dropna(axis=0).values.T
    
    Y = len(tempSec[0][0:X]) 

    topXsectors.loc[0:(Y-1),('Top {}'.format(X),sector)] = tempSec[0][0:Y]

In [10]:
topXsectors

Top 5                                                 \
  Healthcare Basic Materials Consumer Defensive Industrials   
0        JNJ             BHP                WMT         UNP   
1        NVS             LIN                 KO          BA   
2        PFE             ECL                PEP         LMT   
3       ABBV             APD               COST         UPS   
4        ABT             SHW                FMX         RTX   

                                                                         \
  Financial Services Technology Consumer Cyclical Real Estate Utilities   
0                  V       AAPL              BABA         AMT         D   
1                 MA       MSFT              TSLA         CCI        SO   
2               PYPL        TSM                TM         PLD       AEP   
3                BAC       INTC               NKE         PSA       EXC   
4                LFC       NVDA               MCD        CSGP       SRE   

                                           
  Communication Services Energy Financial  
0                   GOOG    PTR       IIM  
1                     FB     BP       ZTR  
2                      T    SNP      OTTW  
3                    DIS    ENB       NaN  
4                    CHL    PBR       NaN

In [11]:
topXsectors.columns = topXsectors.columns.droplevel(0)

In [12]:
topXsectors

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,JNJ,BHP,WMT,UNP,V,AAPL,BABA,AMT,D,GOOG,PTR,IIM
1,NVS,LIN,KO,BA,MA,MSFT,TSLA,CCI,SO,FB,BP,ZTR
2,PFE,ECL,PEP,LMT,PYPL,TSM,TM,PLD,AEP,T,SNP,OTTW
3,ABBV,APD,COST,UPS,BAC,INTC,NKE,PSA,EXC,DIS,ENB,NaN
4,ABT,SHW,FMX,RTX,LFC,NVDA,MCD,CSGP,SRE,CHL,PBR,NaN


In [13]:
topXsectors.columns!='Financial'

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False])

In [14]:
# List for data extraction
topXsectors.loc[:,topXsectors.columns != 'Financial'].values.flatten()

array(['JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
       'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
       'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
       'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
       'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
       'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR'],
      dtype=object)

In [15]:
stockInfo.head()

date          2020-07-05                       
  ticker              sector exchange     marketCap
0      A          Healthcare      NYQ  2.738234e+10
1     AA     Basic Materials      NYQ  2.039532e+09
2   AAAU                 NaN      NaN           NaN
3   AACG  Consumer Defensive      NGM  4.157730e+07
4   AADR                 NaN      NaN           NaN

In [16]:
'IYZ' in stockInfo.loc[:,('date','ticker')].values

True

In [17]:
sectors

array(['Healthcare', 'Basic Materials', 'Consumer Defensive',
       'Industrials', 'Financial Services', 'Technology',
       'Consumer Cyclical', 'Real Estate', 'Utilities',
       'Communication Services', 'Energy', 'Financial'], dtype=object)

In [18]:
## IShares Sector ETFS

# iShares Dow Jones U.S. Basic Materials Index:     IYM
# iShares Dow Jones U.S. Consumer Goods Index:      IYK
# iShares Dow Jones U.S. Consumer Services Index:   IYC
# iShares Dow Jones U.S. Energy Index:              IYE
# iShares Dow Jones U.S. Financial Sector Index:    IYF
# iShares Dow Jones U.S. Financial Services Index:  IYG
# iShares Dow Jones U.S. Healthcare Index:          IYH
# iShares Dow Jones U.S. Industrial Index:          IYJ
# iShares Dow Jones U.S. Real Estate Index:         IYR
# iShares Dow Jones U.S. Technology Index:          IYW
# iShares Dow Jones U.S. Telecommunications Index:  IYZ
# iShares Dow Jones Transportation Average Index:   IYT
# iShares Dow Jones U.S. Utilities Index:           IDU
# iShares Cohen & Steers Realty Majors Index:       ICF

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF']
# {i:j for i,j in zip(sectors,etfs)}
pd.DataFrame({i:j for i,j in zip(sectors,etfs)},index=[0])

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,IYH,IYM,IYK,IYJ,IYG,IYW,IYC,IYR,IDU,IYZ,IYE,IYF


## Let's extract some data now!

In [53]:
all_tickers = [
#                 'JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
#                'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
#                'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
#                'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
#                'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
#                'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR',
#                'IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE', 
               'DIA', 'QQQ', 'SPY', 'XNTK'
              ]

In [23]:
import os
path = 'f:/taqhdf5'
allFiles = os.listdir(path)
relevantFiles = np.array(allFiles)[index[0]]
quote = [ele for ele in relevantFiles if 'quote' in ele]
quote

NameError: name 'index' is not defined

In [26]:


raw_data = h5py.File('F:/TAQHDF5/taqquote_20200501.h5','r')

# Reading one file at a time
# raw_data = h5py.File(path+'/'+file,'r')
dt2 = raw_data['Quotes'].dtype
# if verbose:
#     print('3 Lap time: %.3f' % ((time.time()-start)))


# Store the trade indecies
QI = raw_data['QuoteIndex']

# if verbose:
#     print('4 Lap time: %.3f' % ((time.time()-start)))
# if (verbose) & (i==0):
print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

# Extracting just the tickers
QIC = np.array([ele[0].astype(str).strip() for ele in QI])

The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 



In [48]:
sum(QIC == 'QQQ')

1

In [36]:
pos_start = []
pos_range = []
# Lets get data on each ticker for the file processed at the moment
#for j,ticker in enumerate(tickers):
ticker = 'SPY'
tickerInfo = QI[QIC==ticker][0]
pos_start.append(tickerInfo[1])
pos_range.append(tickerInfo[2])
tickerInfo

(b'SPY              ', 854808212, 20744278)

In [37]:
# use boolean mask to slice all at once
selector = zip(pos_start, pos_range)
mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)

for t,(pos_s, pos_r) in zip(tickers,selector):
    mask[pos_s : pos_s + pos_r] = True

tempData = raw_data['Quotes'][mask]

In [39]:
quoteData = pd.DataFrame(tempData, columns= dt2.names)

In [42]:
quoteData.ex.value_counts()

b'K'    3480387
b'Z'    3098428
b'T'    3092973
b'P'    2923827
b'Y'    2336580
b'J'    1854798
b'X'    1282535
b'N'     806124
b'V'     592544
b'B'     486761
b'C'     398865
b'A'     390188
b'M'        268
Name: ex, dtype: int64

In [50]:
pos_start = []
pos_range = []
# Lets get data on each ticker for the file processed at the moment
#for j,ticker in enumerate(tickers):
ticker = 'DIA'
tickerInfo = QI[QIC==ticker][0]
pos_start.append(tickerInfo[1])
pos_range.append(tickerInfo[2])
tickerInfo

# use boolean mask to slice all at once
selector = zip(pos_start, pos_range)
mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)

for t,(pos_s, pos_r) in zip(tickers,selector):
    mask[pos_s : pos_s + pos_r] = True

tempData = raw_data['Quotes'][mask]

quoteData = pd.DataFrame(tempData, columns= dt2.names)
quoteData.ex.value_counts()

b'Z'    857974
b'P'    857555
b'T'    801273
b'K'    670166
b'B'    418169
b'N'    397281
b'J'    330031
b'Y'    327987
b'X'    324969
b'C'    105533
b'V'     39353
b'A'      7591
b'M'        40
Name: ex, dtype: int64

In [51]:
pos_start = []
pos_range = []
# Lets get data on each ticker for the file processed at the moment
#for j,ticker in enumerate(tickers):
ticker = 'QQQ'
tickerInfo = QI[QIC==ticker][0]
pos_start.append(tickerInfo[1])
pos_range.append(tickerInfo[2])
tickerInfo

# use boolean mask to slice all at once
selector = zip(pos_start, pos_range)
mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)

for t,(pos_s, pos_r) in zip(tickers,selector):
    mask[pos_s : pos_s + pos_r] = True

tempData = raw_data['Quotes'][mask]

quoteData = pd.DataFrame(tempData, columns= dt2.names)
quoteData.ex.value_counts()

b'Z'    2596861
b'K'    2436671
b'Q'    2326518
b'P'    2092825
b'J'    1461334
b'Y'    1224991
b'X'     844870
b'N'     665121
b'B'     302926
b'C'     130412
b'V'     112953
b'A'      58806
b'M'        240
Name: ex, dtype: int64

In [52]:
pos_start = []
pos_range = []
# Lets get data on each ticker for the file processed at the moment
#for j,ticker in enumerate(tickers):
ticker = 'XNTK'
tickerInfo = QI[QIC==ticker][0]
pos_start.append(tickerInfo[1])
pos_range.append(tickerInfo[2])
tickerInfo

# use boolean mask to slice all at once
selector = zip(pos_start, pos_range)
mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)

for t,(pos_s, pos_r) in zip(tickers,selector):
    mask[pos_s : pos_s + pos_r] = True

tempData = raw_data['Quotes'][mask]

quoteData = pd.DataFrame(tempData, columns= dt2.names)
quoteData.ex.value_counts()

b'X'    122622
b'J'    117453
b'P'     96505
b'T'     85238
b'Z'     82942
b'K'     52712
b'N'     18695
b'B'     10124
b'Y'      9278
b'C'      7960
b'A'      7748
Name: ex, dtype: int64

In [54]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501',
                  '20200504',
                  '20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529']).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

exchanges = ['z','x'] #['q','t']
select_largest_exchange = True

# run load_data()
candles = load_data_and_save_v2(dates,
                                  tickers,
                                  dataNeeded,
                                  path,
                                  verbose,
                                  extract_candles,
                                  aggHorizon,
                                  extra_features_from_quotes,
                                  data_sample,
                                  exchanges,
                                  save_output,
                                  select_largest_exchange)

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200529


1 Lap time: 0.000

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.000

### Quote Data ###

3 Lap time: 0.001
4 Lap time: 0.001
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.887
6 Lap time: 0.889
7 Lap time: 0.902
8 Lap time: 60.673
9 Lap time: 72.393
10 Lap time: 84.058
11 Lap time: 84.358
12 Lap time: 88.117
Step: 0.16666666666666666 -- Ticker: DIA -- Exchange sizes are: ex
X    

Step: 0.5 -- Ticker: XNTK -- Exchange sizes are: ex
X    32776
Z    29791
dtype: int64
Step: 1 -- Ticker: DIA -- Exchange sizes are: ex
X    119643
Z    353385
dtype: int64
Step: 1 -- Ticker: QQQ -- Exchange sizes are: ex
X     535149
Z    1393211
dtype: int64
Step: 1 -- Ticker: SPY -- Exchange sizes are: ex
X     616935
Z    1715153
dtype: int64
Step: 1 -- Ticker: XNTK -- Exchange sizes are: ex
X    32776
Z    29791
dtype: int64
3 Lap time: 847.652
4 Lap time: 847.653
5 Lap time: 848.603
6 Lap time: 848.606
7 Lap time: 848.708
8 Lap time: 893.071
9 Lap time: 900.613
10 Lap time: 908.091
11 Lap time: 908.280
12 Lap time: 910.718
Step: 0.16666666666666666 -- Ticker: DIA -- Exchange sizes are: ex
X    140145
Z    487108
dtype: int64
Step: 0.16666666666666666 -- Ticker: QQQ -- Exchange sizes are: ex
X     435139
Z    1974404
dtype: int64
Step: 0.16666666666666666 -- Ticker: SPY -- Exchange sizes are: ex
X     480474
Z    1967722
dtype: int64
Step: 0.16666666666666666 -- Ticker: XNTK -- Ex

Step: 1 -- Ticker: SPY -- Exchange sizes are: ex
X    1051065
Z    2366591
dtype: int64
Step: 1 -- Ticker: XNTK -- Exchange sizes are: ex
X    96369
Z    44919
dtype: int64
3 Lap time: 2077.948
4 Lap time: 2077.950
5 Lap time: 2078.899
6 Lap time: 2078.901
7 Lap time: 2079.048
8 Lap time: 2179.710
9 Lap time: 2189.769
10 Lap time: 2199.823
11 Lap time: 2200.078
12 Lap time: 2203.427
Step: 0.16666666666666666 -- Ticker: DIA -- Exchange sizes are: ex
X    156416
Z    745965
dtype: int64
Step: 0.16666666666666666 -- Ticker: QQQ -- Exchange sizes are: ex
X     504706
Z    1796426
dtype: int64
Step: 0.16666666666666666 -- Ticker: SPY -- Exchange sizes are: ex
X     518614
Z    2653625
dtype: int64
Step: 0.16666666666666666 -- Ticker: XNTK -- Exchange sizes are: ex
X    126324
Z     52836
dtype: int64
Step: 0.5 -- Ticker: DIA -- Exchange sizes are: ex
X    156416
Z    745965
dtype: int64
Step: 0.5 -- Ticker: QQQ -- Exchange sizes are: ex
X     504706
Z    1796426
dtype: int64
Step: 0.5 -- Ti

5 Lap time: 3364.069
6 Lap time: 3364.072
7 Lap time: 3364.235
8 Lap time: 3532.211
9 Lap time: 3544.034
10 Lap time: 3555.822
11 Lap time: 3556.118
12 Lap time: 3559.965
Step: 0.16666666666666666 -- Ticker: DIA -- Exchange sizes are: ex
X    141902
Z    835111
dtype: int64
Step: 0.16666666666666666 -- Ticker: QQQ -- Exchange sizes are: ex
X     773824
Z    2019269
dtype: int64
Step: 0.16666666666666666 -- Ticker: SPY -- Exchange sizes are: ex
X     806755
Z    2875954
dtype: int64
Step: 0.16666666666666666 -- Ticker: XNTK -- Exchange sizes are: ex
X    127529
Z     70308
dtype: int64
Step: 0.5 -- Ticker: DIA -- Exchange sizes are: ex
X    141902
Z    835111
dtype: int64
Step: 0.5 -- Ticker: QQQ -- Exchange sizes are: ex
X     773824
Z    2019269
dtype: int64
Step: 0.5 -- Ticker: SPY -- Exchange sizes are: ex
X     806755
Z    2875954
dtype: int64
Step: 0.5 -- Ticker: XNTK -- Exchange sizes are: ex
X    127529
Z     70308
dtype: int64
Step: 1 -- Ticker: DIA -- Exchange sizes are: ex
X 

In [21]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529'
                  
                  
                 ]).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

# run load_data()
candles = load_data_final(dates,
                          tickers,
                          dataNeeded,
                          path,
                          verbose,
                          extract_candles,
                          aggHorizon,
                          extra_features_from_quotes,
                          data_sample,
                          save_output)

dates,
                    tickers,
                    dataNeeded,
                    path,
                    verbose,
                    extract_candles = False,
                    aggHorizon = 1,
                    extra_features_from_quotes = None,
                    data_sample = 'full',
                    exchanges = ['q','t'],
                    save_output = False):

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200505
Date, Max: 20200529


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.002

### Quote Data ###

3 Lap time: 0.076
4 Lap time: 0.076
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.868
6 Lap time: 0.878
7 Lap time: 62.353
8 Lap time: 77.898
9 Lap time: 92.528
10 Lap time: 107.148
11 Lap time: 107.512
12 Lap time: 112.094
3 Lap time: 353.492
4 Lap time: 353.493
5 Lap time: 354.299
6 Lap ti

TypeError: unsupported operand type(s) for /: 'list' and 'int'